## Generating SQL with Python

The metadata for the database describing the tables present, and their columns, is defined in Python using SQLAlchemy, the leading python database tool, thus:

In [ ]:
import sqlalchemy
from sqlalchemy import text

import os
try:
    os.remove('molecules.db')
    print("Remove database to teach again from scratch")
except FileNotFoundError:
    print("No DB since this notebook was last run")

engine = sqlalchemy.create_engine('sqlite:///molecules.db', echo=True)
    
from sqlalchemy import Table, Column, Integer, Float, String, MetaData, ForeignKey
metadata = MetaData()
molecules = Table('molecules', metadata,
                  Column('name', String, primary_key=True),
                  Column('mass', Float))

atoms = Table('atoms', metadata,
      Column('symbol', String, primary_key=True),
      Column('number', Integer)
             )

In [ ]:
atoms_in_molecules = Table('atoms_molecules', metadata,
       Column('atom', None, ForeignKey('atoms.symbol')),
       Column('molecule', None, ForeignKey('molecules.name')),
       Column('number', Integer)
)

In [ ]:
metadata.create_all(engine)
print(metadata)

Note the SQL syntax for creating tables is generated by the python tool, and sent to the database server.

```
CREATE TABLE molecules (
	name VARCHAR NOT NULL, 
	mass FLOAT, 
	PRIMARY KEY (name)
)
```

We'll turn off our automatic printing of all the raw sql to avoid this notebook being unreadable.

In [ ]:
engine.echo=False

We can also write data to our database using this python tooling:

In [ ]:
ins = molecules.insert().values(name='water', mass='18.01')

In [ ]:
conn = engine.connect()
conn.execute(ins)

And query it:

In [ ]:
from sqlalchemy.sql import select
s = select(molecules)
result = conn.execute(s)

In [ ]:
print(result.fetchone())

If we have enough understanding of sql syntax, we can use appropriate **join** statements to find, for example, the mass of all molecules which contain oxygen:

In [ ]:
conn.execute(molecules.insert().values(name='oxygen', mass='16.00'))
conn.execute(atoms.insert().values(symbol='O', number=8))
conn.execute(atoms.insert().values(symbol='H', number=1))
conn.execute(atoms_in_molecules.insert().values(molecule='water',atom='O',number=1))
conn.execute(atoms_in_molecules.insert().values(molecule='oxygen',atom='O',number=2))
conn.execute(atoms_in_molecules.insert().values(molecule='water',atom='H', number=1))

In [ ]:
result=conn.execute(text(
    'SELECT mass FROM molecules JOIN atoms_molecules ' + 
    'ON molecules.name = atoms_molecules.molecule JOIN atoms ' +
    'ON atoms.symbol = atoms_molecules.atom ' +
    'WHERE atoms.symbol = "H"'))
print(result.fetchall())

But we can do much better...